Step 1: Preparation 

Before you begin, make sure that you have set the runtime type to GPU  (Hardware acclerator: GPU).

In [2]:
ROOTPATH="/content/ailab/" # root dir (a constant)
ModelPath=ROOTPATH+"Real-CUGAN/model/" # model dir
PendingPath=ROOTPATH+"Real-CUGAN/pending/" # input dir
FinishPath=ROOTPATH+"Real-CUGAN/finish/" # output dir
ModelName="up2x-latest-no-denoise.pth" # default model
Tile=4 #{0,1,2,3,4,auto}; the larger the number, the smaller the memory consumption

# initialize environment
!pip install torch opencv-python
!git clone https://github.com/bilibili/ailab.git
from google.colab import drive
drive.mount('/content/gdrive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'ailab'...
remote: Enumerating objects: 402, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 402 (delta 22), reused 41 (delta 21), pack-reused 355
Receiving objects: 100% (402/402), 14.79 MiB | 34.04 MiB/s, done.
Resolving deltas: 100% (148/148), done.
Mounted at /content/gdrive


Step 2: Download model files from [here](https://drive.google.com/drive/folders/1UFgpV14uEAcgYvVw0fJuajzy1k7JIz6H) and save them - in a folder called `updated_weights` - under your Google Drive's root folder.

Step 3: Put/upload your image(s) under `/content/aliab/Real-CUGAN/pending`.

Then, run the following and choose the model you want.

In [ ]:
import os
os.mkdir(ModelPath)
os.mkdir(PendingPath)
os.mkdir(FinishPath)
!cp -r /content/gdrive/MyDrive/updated_weights/* /content/ailab/Real-CUGAN/model/
fileNames = os.listdir(PendingPath)
print("Pending images:")
for i in fileNames:
  print("\t"+i)
fileNames = os.listdir(ModelPath)
print("Model files available:")
for idx, i in enumerate(fileNames):
  print(f"{idx+1}. \t {i}")
choice = int(input("Select model (leave blank for default)："))
if choice:
  ModelName=fileNames[choice-1]
Amplification=ModelName[2] # amplifying ratio
if (not os.path.isfile(ModelPath+ModelName)):
  print("Warning: selected model file does not exist")

Pending images:
Model files available:
1. 	 up4x-latest-conservative.pth
2. 	 up3x-latest-no-denoise.pth
3. 	 up3x-latest-conservative.pth
4. 	 up2x-latest-denoise3x.pth
5. 	 up2x-latest-conservative.pth
6. 	 up3x-latest-denoise3x.pth
7. 	 up2x-latest-denoise1x.pth
8. 	 up4x-latest-denoise3x.pth
9. 	 up2x-latest-no-denoise.pth
10. 	 up4x-latest-no-denoise.pth
11. 	 up2x-latest-denoise2x.pth


In [4]:
import shutil
import os
 
source_file = "/content/gdrive/MyDrive/zpk.zip.zip"
target_dir = "/content/ailab/Real-CUGAN/pending"
 
shutil.unpack_archive(source_file, target_dir)

In [5]:
torch.cuda.empty_cache()

NameError: ignored

Step 4: Run the processing script.

In [ ]:
import sys
sys.path.append("/content/ailab/Real-CUGAN")
import matplotlib.pyplot as plt
%matplotlib inline
import torch
from torch import nn as nn
from torch.nn import functional as F
import os,sys,cv2
import numpy as np
from upcunet_v3 import RealWaifuUpScaler
from time import time as ttime  
fileNames = os.listdir(PendingPath)
print(f"using model {ModelPath+ModelName}")
upscaler = RealWaifuUpScaler(2, ModelPath+ModelName, half=True, device="cuda:0")
t0 = ttime()
for i in fileNames:
  torch.cuda.empty_cache()
  try:
    img = cv2.imread(PendingPath+i)[:, :, [2, 1, 0]]
    result = upscaler(img,tile_mode=5,cache_mode=2,alpha=1)
    cv2.imwrite(FinishPath+i,result[:, :, ::-1])
  except RuntimeError as e:
    print (i+" FAILED")
    print (e)
  else:
    print(i+" DONE")
    os.remove(PendingPath+i)
t1 = ttime()
print("time_spent", t1 - t0)

In [7]:
import shutil
 
shutil.rmtree('/content/gdrive/MyDrive/finish')

In [8]:
import os 
import zipfile
  
zip_file = zipfile.ZipFile('/content/finish.zip', 'w') 
 
for root, dirs, files in os.walk('/content/ailab/Real-CUGAN/finish'):
  for file in files:
    zip_file.write(os.path.join(root, file))
 
zip_file.close()

In [9]:
import os
import shutil

shutil.move('/content/finish.zip', '/content/gdrive/MyDrive')
 
file_stats = os.stat('/content/gdrive/MyDrive/finish.zip')
file_size = file_stats.st_size
 
print(f'finish.zip file size: {file_size} bytes')

finish.zip file size: 7747949077 bytes


In [16]:
!ffmpeg -threads 16 -r 23.976 -f image2 -s 1920x1080 -i /content/ailab/Real-CUGAN/finish/%05N.png -t 46 -c:v libx264 -crf 25  -pix_fmt yuv420p /content/video.mp4

^C


In [19]:
!pip install ffmpeg-python
 
import ffmpeg
import os
 

img_folder = '/content/ailab/Real-CUGAN/finish' 

video_file = '/content/video.mp4'
 

img_files = os.listdir(img_folder) 
img_files.sort()
 
  
input_streams = [ffmpeg.input(img_folder + img_file) for img_file in img_files]  
 

joined_streams = ffmpeg.concat(*input_streams,v=1,a=1)
 

output_opts = {
    'c:v': 'libx264',
    'r': '23.976', 
    'pix_fmt': 'yuv420p',
    'b:v': '4096k',
    'preset': 'ultrafast',
    'tune': 'zerolatency',
    'threads':'16'
}
 
  
output = ffmpeg.output(joined_streams, video_file, **output_opts)  
 
 
ffmpeg.run(output, overwrite_output=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


ValueError: ignored

In [20]:
import os

dir_path = '/content/ailab/Real-CUGAN/finish'

num_files = len(os.listdir(dir_path))

print(f'There are {num_files} files in {dir_path}')

There are 1431 files in /content/ailab/Real-CUGAN/finish


In [ ]:
你可以使用unzip命令解压zip文件: bash
 unzip /content/gdrive/MyDrive/finish.zip -d /content/ailab/Real-CUGAN/finish


这个命令的意思是:

- unzip: 调用unzip命令解压zip文件
- /content/gdrive/MyDrive/finish.zip: 待解压的zip文件路径
- -d /content/ailab/Real-CUGAN/finish: 将文件解压到此目录下

所以完整的命令是: bash 
 

执行此命令后,zip文件中的所有内容会被解压到 /content/ailab/Real-CUGAN/finish目录下。

如果解压过程中出现任何错误,请检查:

1. zip文件路径是否正确
2. 目标解压目录是否存在,不存在的话需要先创建目录
3. 你是否有足够的权限解压到目标目录

遇到问题可以在这里回复,我会尽力帮助你解决。

In [ ]:
!unzip /content/gdrive/MyDrive/finish.zip -d /content/ailab/Real-CUGAN/finish

In [37]:
import os

dir_path = '/content/ailab/Real-CUGAN/finish'
files = os.listdir(dir_path)
files.sort()

ffmpeg_command = [
    'ffmpeg',
    '-framerate', '23.976',
    '-i', os.path.join(dir_path, '%05d.png'),
    '-vcodec', 'libx264', 
    '-b:v', '50M',
    '-crf', '16',
    '-pix_fmt', 'yuv420p',
    '-r', '23.976',
    '-threads', '16',
    os.path.join(dir_path, '/content/output.mp4')
]

os.system(' '.join(ffmpeg_command))

print('Video created!')

Video created!


In [1]:
import os

dir_path = '/content/ailab/Real-CUGAN/finish'
files = os.listdir(dir_path)
files.sort()

ffmpeg_command = [
    'ffmpeg',
    '-framerate', '23.976',
    '-i', os.path.join(dir_path, '%05d.png'),
    '-c:v', 'prores_ks',
    '-profile:v', '3',
    '-pix_fmt', 'yuv420p10le',
    '-r', '23.976',
    '-threads', '16',
    os.path.join(dir_path, '/content/output.mov')
]

os.system(' '.join(ffmpeg_command))

print('Video created!')

FileNotFoundError: ignored

In [ ]:
要使用无损质量和高质量编码合并图片为视频,可以使用以下的ffmpeg命令:

 ffmpeg_command = [
     'ffmpeg',
     '-framerate', '23.976',
     '-i', os.path.join(dir_path, '%05d.png'),
     '-c:v', 'prores_ks', 
     '-profile:v', '3',  
     '-pix_fmt', 'yuv422p10le',
     '-r', '23.976',
     'threads', '16',
     os.path.join(dir_path, '/content/output.mov')
 ] 


这些选项的意思是:

- -c:v prores_ks : 使用 Apple ProRes 编解码器,无损压缩,高质量
- -profile:v 3 : 使用 ProRes 422 HQ 配置文件,略有压缩但质量很高 
- -pix_fmt yuv422p10le : 10 位精度,高动态范围和色度
- 使用 .mov 作为输出格式,更适合这种高质量视频

使用这些设置,你可以将图片无损合成为一个高质量的视频。 ProRes 是一个专业级的视频编码格式,质量非常高,所以输出的视频质量会很好。

另外,你提到图片是从 m2ts 视频抽取的,那么视频的分辨率和帧率信息你应该已经有。所以可以直接使用,不需要 ffmpeg 重新检测视频信息。这可以加快处理速度。

Hope this helps! Let me know if you have any other questions.

In [ ]:
可以使用以下命令显示 /content/output.mp4 文件大小(以MB为单位):

 bash
 stat -c%s /content/output.mp4


这个命令会输出文件的大小(以字节为单位)。

然后我们可以计算并格式化为MB:

 bash 
 


这个表达式会将文件大小除以1024(转换为KB),再除以1024(转换为MB),并显示结果,加上MB字符串。

所以整个命令是:

 bash 
 filesize=$(stat -c%s /content/output.mp4) 
 echo $((filesize/1024/1024))MB


例如,如果 output.mp4 文件大小是243000000字节,上面的命令会输出:

 232MB


 human: 显示视频文件output.mp4的视频时长

In [34]:
!filesize=$(stat -c%s /content/output.mp4)
!echo $((filesize/1024))MB

0MB


In [11]:

!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
要为 /content/gdrive/MyDrive/finish.zip 创建一份备份,你可以:

1. 复制该文件,创建一个副本:

 python
 import shutil
 
 source_dir = '/content/gdrive/MyDrive/finish.zip'
 dest_dir = '/content/gdrive/MyDrive/'
 dest_file = 'finish_backup.zip'
 
 shutil.copy(source_dir, dest_dir)


这会在 /content/gdrive/MyDrive/ 下创建一个 finish_backup.zip 的备份文件。

2. 也可以使用 shutil 创建一个压缩文件副本:

 python 



这会在 /content/gdrive/MyDrive/backup/ 下创建一个名为 finish.zip 的备份文件。

3. 更高级的方法是使用 shutil 的 archiving 功能直接创建一个 zip 或 tar 归档文件:

 python
 import shutil
 
 source_dir = '/content/gdrive/MyDrive/finish.zip' 
 dest_dir = '/content/gdrive/MyDrive/backup/'
 dest_archive = 'finish_backup.zip'
 
 shutil.make_archive(os.path.join(dest_dir, dest_archive), 'zip', source_dir)


这会创建一个zip归档文件 /content/gdrive/MyDrive/backup/finish_backup.zip,其中包含 finish.zip 的所有内容。

所以以上三种方法任选一种,都可以为 finish.zip 创建一份完整的备份。请 let 我知道如果有任何其他问题!

In [28]:
import shutil
 
source_dir = '/content/gdrive/MyDrive/finish.zip'
dest_dir = '/content/gdrive/MyDrive/backup/'
 
shutil.copy(source_dir, dest_dir)

'/content/gdrive/MyDrive/backup/finish.zip'

SameFileError: ignored

Generated images should be under /content/aliab/Real-CUGAN/finish.